## Techniques to interact with a Database

### Technique 1 :
* Using packages like pyodbc, pymssql directly

In [1]:
import pyodbc

In [2]:
server = 'faa-sql.database.windows.net'
database = 'faa-airline'
username = 'faa-admin'
password = 'fractal$1234'
driver= '{ODBC Driver 17 for SQL Server}'
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
cursor.execute("select top 5* from dbo.trips")       
for row in cursor:
    print(row)

(4118, datetime.datetime(2014, 12, 29, 13, 52), datetime.datetime(2014, 12, 29, 13, 56), 2180, 224, 283, 287, 'Subscriber', 'Female', 1956)
(4275, datetime.datetime(2014, 12, 29, 13, 53), datetime.datetime(2014, 12, 29, 14, 0), 75, 430, 81, 66, 'Subscriber', 'Male', 1991)
(4291, datetime.datetime(2014, 12, 29, 13, 54), datetime.datetime(2014, 12, 29, 14, 0), 2398, 354, 66, 75, 'Subscriber', 'Male', 1979)
(4316, datetime.datetime(2014, 12, 29, 13, 54), datetime.datetime(2014, 12, 29, 14, 0), 2024, 326, 77, 81, 'Subscriber', 'Male', 1970)
(4342, datetime.datetime(2014, 12, 29, 13, 54), datetime.datetime(2014, 12, 29, 13, 57), 2474, 205, 176, 291, 'Subscriber', 'Male', 1984)


### Technique 2 :
* using sqlalchemy package which internally uses pyodbc or pymssql package

In [9]:
import sqlalchemy as sa
import urllib.parse
from sqlalchemy import create_engine

params = urllib.parse.quote_plus("DRIVER={SQL Server};SERVER=faa-sql.database.windows.net;DATABASE=faa-airline;UID=faa-admin@faa-sql;PWD=fractal$1234")
print(params)
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" %params)
metadata = sa.MetaData()

DRIVER%3D%7BSQL+Server%7D%3BSERVER%3Dfaa-sql.database.windows.net%3BDATABASE%3Dfaa-airline%3BUID%3Dfaa-admin%40faa-sql%3BPWD%3Dfractal%241234


In [10]:
connection = engine.connect()

In [11]:
trips = sa.Table('trips', metadata, autoload=True, autoload_with=engine)

In [12]:
type(trips)

sqlalchemy.sql.schema.Table

In [14]:
# Print the column names
print(trips.columns.keys())

['trip_id', 'starttime', 'stoptime', 'bikeid', 'tripduration', 'from_station_id', 'to_station_id', 'usertype', 'gender', 'birthday']


In [15]:
# Print full table metadata
print(repr(metadata.tables['trips']))

Table('trips', MetaData(bind=None), Column('trip_id', INTEGER(), table=<trips>, primary_key=True, nullable=False), Column('starttime', DATETIME2(), table=<trips>), Column('stoptime', DATETIME2(), table=<trips>), Column('bikeid', INTEGER(), table=<trips>), Column('tripduration', INTEGER(), table=<trips>), Column('from_station_id', INTEGER(), table=<trips>), Column('to_station_id', INTEGER(), table=<trips>), Column('usertype', NVARCHAR(length=50), table=<trips>), Column('gender', NVARCHAR(length=50), table=<trips>), Column('birthday', INTEGER(), table=<trips>), schema=None)


In [16]:
#Equivalent to 'SELECT * FROM store_details'
query = sa.select([trips])

In [17]:
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()

In [18]:
ResultSet

[(4118, '2014-12-29 13:52:00.0000000', '2014-12-29 13:56:00.0000000', 2180, 224, 283, 287, 'Subscriber', 'Female', 1956),
 (4275, '2014-12-29 13:53:00.0000000', '2014-12-29 14:00:00.0000000', 75, 430, 81, 66, 'Subscriber', 'Male', 1991),
 (4291, '2014-12-29 13:54:00.0000000', '2014-12-29 14:00:00.0000000', 2398, 354, 66, 75, 'Subscriber', 'Male', 1979),
 (4316, '2014-12-29 13:54:00.0000000', '2014-12-29 14:00:00.0000000', 2024, 326, 77, 81, 'Subscriber', 'Male', 1970),
 (4342, '2014-12-29 13:54:00.0000000', '2014-12-29 13:57:00.0000000', 2474, 205, 176, 291, 'Subscriber', 'Male', 1984),
 (4480, '2014-12-29 13:54:00.0000000', '2014-12-29 13:59:00.0000000', 485, 332, 137, 241, 'Subscriber', 'Male', 1984),
 (4490, '2014-12-29 13:55:00.0000000', '2014-12-29 14:22:00.0000000', 854, 1566, 85, 296, 'Subscriber', 'Female', 1988),
 (4592, '2014-12-29 13:55:00.0000000', '2014-12-29 14:06:00.0000000', 1502, 644, 134, 36, 'Subscriber', 'Male', 1982),
 (4602, '2014-12-29 13:55:00.0000000', '2014-12

In [19]:
# import pandas
import pandas as pd
df = pd.DataFrame(ResultSet)
df.columns = ResultSet[0].keys()

In [20]:
df.columns

Index(['trip_id', 'starttime', 'stoptime', 'bikeid', 'tripduration',
       'from_station_id', 'to_station_id', 'usertype', 'gender', 'birthday'],
      dtype='object')

### Technique 3 : 
* Using Jupyter SQL magic function

In [22]:
# import ipython-sql package if not installed
!pip install ipython-sql
# load magic sql
%load_ext sql

In [23]:
%sql mssql+pyodbc:///?odbc_connect=DRIVER%3D%7BSQL+Server%7D%3BSERVER%3Dfaa-sql.database.windows.net%3BDATABASE%3Dfaa-airline%3BUID%3Dfaa-admin%40faa-sql%3BPWD%3Dfractal%241234

'Connected: @'

In [24]:
%%sql
select top 5* from trips

 * mssql+pyodbc:///?odbc_connect=DRIVER={SQL Server};SERVER=faa-sql.database.windows.net;DATABASE=faa-airline;UID=faa-admin@faa-sql;PWD=fractal$1234
Done.


trip_id,starttime,stoptime,bikeid,tripduration,from_station_id,to_station_id,usertype,gender,birthday
4118,2014-12-29 13:52:00.0000000,2014-12-29 13:56:00.0000000,2180,224,283,287,Subscriber,Female,1956
4275,2014-12-29 13:53:00.0000000,2014-12-29 14:00:00.0000000,75,430,81,66,Subscriber,Male,1991
4291,2014-12-29 13:54:00.0000000,2014-12-29 14:00:00.0000000,2398,354,66,75,Subscriber,Male,1979
4316,2014-12-29 13:54:00.0000000,2014-12-29 14:00:00.0000000,2024,326,77,81,Subscriber,Male,1970
4342,2014-12-29 13:54:00.0000000,2014-12-29 13:57:00.0000000,2474,205,176,291,Subscriber,Male,1984


In [25]:
%%sql data <<
select gender,count(trip_id) as trips
from dbo.trips
group by gender

 * mssql+pyodbc:///?odbc_connect=DRIVER={SQL Server};SERVER=faa-sql.database.windows.net;DATABASE=faa-airline;UID=faa-admin@faa-sql;PWD=fractal$1234
Done.
Returning data to local variable data


In [26]:
type(data)

sql.run.ResultSet

In [27]:
import pandas as pd
df = pd.DataFrame(data,columns=['Gender','Trips'])

In [28]:
df.head()

,Gender,Trips
0,None,12628
1,Male,24660
2,Female,7695
